In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import key_dict

In [2]:
bitcoin = pd.read_csv('bitcoin all time.csv')
dow_jones = pd.read_csv('HistoricalDJIA.csv')
goldprices_df = pd.read_csv("Gold Prices 1979-2021.csv")

In [3]:
bitcoin = bitcoin[['Date', 'Closing Price (USD)']]
dow_jones = dow_jones[['Date', ' Close']]
goldprices_clean_df = goldprices_df[['date', 'price in usd per troy oz']].copy()

bitcoin.Date = pd.to_datetime(bitcoin.Date)
dow_jones.Date = pd.to_datetime(dow_jones.Date)
goldprices_clean_df.date = pd.to_datetime(goldprices_clean_df.date)

bitcoin.reset_index(inplace = True, drop = True)
dow_jones.reset_index(inplace = True, drop = True)
goldprices_clean_df.reset_index(inplace = True, drop = True)

bitcoin['id'] = list(bitcoin.index.values)
dow_jones['id'] = list(dow_jones.index.values)
goldprices_clean_df['id'] = list(goldprices_clean_df.index.values)

In [4]:
bitcoin.rename(columns = {"Date":"date_", "Closing Price (USD)":"price_usd_bc"}, inplace = True)
dow_jones.rename(columns = {"Date":"date_", " Close":"score_dj"}, inplace = True)
goldprices_clean_df.rename(columns={"date": "date_",
                                    "price in usd per troy oz":"price_usd_gold"},inplace = True)

In [5]:
goldprices_clean_df.price_usd_gold = goldprices_clean_df.price_usd_gold.replace(",","",regex = True)
goldprices_clean_df.price_usd_gold = goldprices_clean_df.price_usd_gold.astype(float)

In [6]:
goldprices_clean_df

,date_,price_usd_gold,id
0,1978-12-29,226.0,0
1,1979-01-01,226.0,1
2,1979-01-02,226.8,2
3,1979-01-03,218.6,3
4,1979-01-04,223.2,4
...,...,...,...
11036,2021-04-19,1774.5,11036
11037,2021-04-20,1777.9,11037
11038,2021-04-21,1798.2,11038
11039,2021-04-22,1787.8,11039


In [7]:
connection_string = "postgres:" + key_dict['postgres'] + "@localhost:5432/etl-project"
engine = create_engine(f"postgresql://{connection_string}")

In [8]:
engine.table_names()

F:\anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['djia_data', 'gold_prices', 'bitcoin_data']

In [9]:
bitcoin.to_sql(name = 'bitcoin_data', con = engine, if_exists = 'replace', index = False)
dow_jones.to_sql(name = 'djia_data', con = engine, if_exists = 'replace', index = False)
goldprices_clean_df.to_sql(name='gold_prices', con=engine, if_exists='append', index=False)

In [10]:
pd.read_sql_query("SELECT bitcoin_data.date_, bitcoin_data.price_usd_bc, djia_data.score_dj, gold_prices.price_usd_gold \
                    FROM bitcoin_data \
                    JOIN djia_data \
                    ON bitcoin_data.date_ = djia_data.date_ \
                    JOIN gold_prices \
                    ON gold_prices.date_ = djia_data.date_", con = engine)

,date_,price_usd_bc,score_dj,price_usd_gold
0,2013-09-30,123.654990,15129.67,1326.5
1,2013-10-01,125.455000,15191.70,1290.8
2,2013-10-02,108.584830,15133.14,1306.3
3,2013-10-03,118.674660,14996.48,1316.0
4,2013-10-04,121.338660,15072.58,1309.8
...,...,...,...,...
1902,2021-04-19,56224.101588,34077.63,1774.5
1903,2021-04-20,56608.769748,33821.30,1777.9
1904,2021-04-21,54144.427476,34137.31,1798.2
1905,2021-04-22,51965.059559,33815.90,1787.8
